# Anomaly Detection: The Local Outlier Factor (LOF) Model

## Introductory Remarks

Anomalies are data points that are different from other observations in some way, typically measured against a model fit to the data. On the contrary with the ordinary descriptive statistics, we are interested here to found where these anomalous data points exist and not exclude them as outliers.

We assume the anomaly detection task is unsupervised, i.e. we don’t have training data with points labeled as anomalous. Each data point passed to an anomaly detection model is given a score indicating how different the point is relative to the rest of the dataset. The calculation of this score varies between models, but a higher score always indicates a point is more anomalous. Often a threshold is chosen to make a final classification of each point as typical or anomalous; this post-processing step is left to the user.

The **GraphLab Create (GLC) Anomaly Detection toolkit** currently includes three models for two different data contexts: 

* **Local Outlier Factor**, for detecting outliers in multivariate data that are assumed to be independently and identically distributed,
* **Moving Z-score**, for scoring outliers in a univariate, sequential dataset, typically a time series, and
* **Bayesian Changepoints** for identifying changes in the mean or variance of a sequential series.

In this short note, we demonstrate how the **GLC Local Outlier Factor** Model can be used to *reveal anomalies* in a *multivariate data set*. We will use the customer data from a recent [**AirBnB New User Bookings competition**](https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings) on Kaggle. More specifically, we have downloaded a copy of the file **`train_users_2.csv`** in our working directory. Each row in this dataset describes one of 213,451 AirBnB users; there is a mix of basic features, such as **`gender`**, **`age`**, and **`preferred language`**, as well as the **user's "technology profile"**, including the **browser type**, **device type**, and **his/her sign-up method**.

## Libraries and Necessary Data Transformation

First, we fire up **GraphLab Create**, all the other necessary libraries for our study, and load the **`train_users_2.csv`** file in a **SFrame**.

In [1]:
import graphlab as gl

In [2]:
customer_data = gl.SFrame.read_csv('./train_users_2.csv')

2016-05-25 23:07:44,869 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1464206862.log


Finished parsing file /home/theod/Documents/ML_Home/11.Dato_Py/08..Anomaly_Detection/train_users_2.csv

Parsing completed. Parsed 100 lines in 5.03044 secs.

This non-commercial license of GraphLab Create is assigned to tgrammat@gmail.com and will expire on September 21, 2016. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file /home/theod/Documents/ML_Home/11.Dato_Py/08..Anomaly_Detection/train_users_2.csv

Parsing completed. Parsed 213451 lines in 3.9072 secs.


Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,str,str,float,str,int,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
customer_data

id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow
gxn3p5htnn,2010-06-28,20090319043255,,-unknown-,None,facebook,0
820tgsjxq7,2011-05-25,20090523174809,,MALE,38.0,facebook,0
4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3
bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0
87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0
osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,None,basic,0
lsw9q7uk0j,2010-01-02,20100102012558,2010-01-05,FEMALE,46.0,basic,0
0d01nltbrs,2010-01-03,20100103191905,2010-01-13,FEMALE,47.0,basic,0
a1vcnhxeij,2010-01-04,20100104004211,2010-07-29,FEMALE,50.0,basic,0
6uh8zyj2gn,2010-01-04,20100104023758,2010-01-04,-unknown-,46.0,basic,0


For the needs of our current presentation we will only need a small subset of the available basic customer features, i.e. **`'gender'`**, **`'age'`** and **`'language'`**.

In [4]:
features = ['gender', 'age', 'language']
customer_data = customer_data[['id']+features]
customer_data

id,gender,age,language
gxn3p5htnn,-unknown-,None,en
820tgsjxq7,MALE,38.0,en
4ft3gnwmtx,FEMALE,56.0,en
bjjt8pjhuk,FEMALE,42.0,en
87mebub9p4,-unknown-,41.0,en
osr2jwljor,-unknown-,None,en
lsw9q7uk0j,FEMALE,46.0,en
0d01nltbrs,FEMALE,47.0,en
a1vcnhxeij,FEMALE,50.0,en
6uh8zyj2gn,-unknown-,46.0,en


From the quick exploratory data analysis below:

In [5]:
gl.canvas.set_target('browser')
customer_data.show()

Canvas is accessible via web browser at the URL: http://localhost:41189/index.html
Opening Canvas in default web browser.


we notice that there about 750 records having an **`'age'`** value of `'2013'` or `'2014'`, which is of course wrong. Most probably the year was recorded accidentally in this field.  The remaining **`'age'`** values seams absolutely reasonable with only some rare customer entries that have ages greater than `'100'`. In fact more than 128 thousand customer entries are found to have ages in the `[1, 142]` interval. More specifically, we have choosen to assume any value falling in the `[1,150]` interval as an elligible recording of a customer age, re-assigning all the remaining ones as `missing`.

In [6]:
customer_data['age'] = customer_data['age'].apply(lambda age: age if age < 150 else None)
customer_data = customer_data.dropna(columns = features, how='any')
print 'Number of Rows in dataset: %d' % len(customer_data)

Number of Rows in dataset: 124681


In [7]:
gl.canvas.set_target('ipynb')
customer_data.show()

The data set of interest, **customer_data**, has two nominal categorical variables:

* **`'gender'`:** nominal categorical attribute (`FEMALE/MALE/unknown/OTHER`)
* **`'language'`:** nominal categorical attribute of 25 different languages.

which we should better encode them prior of applying any learning algorithm. To do so we will apply the `OneHotEncoding` transformation as shown below:

In [9]:
one_hot_encoder = gl.toolkits.feature_engineering.OneHotEncoder(features=['gender', 'language'])
customer_data2 = one_hot_encoder.fit_transform(customer_data)

Local Outlier Factor (LOF) Models are distance-based learning algorithms. Therefore, we need to standardize the **`'age'`** feature in order to be on roughly the same scale as the encoded categorical variables.

In [10]:
customer_data2['age'] = (customer_data['age'] - customer_data['age'].mean())/customer_data['age'].std()
customer_data2

id,age,encoded_features
820tgsjxq7,0.0420907775098,"{2: 1, 28: 1}"
4ft3gnwmtx,1.33196384402,"{3: 1, 28: 1}"
bjjt8pjhuk,0.328729236733,"{3: 1, 28: 1}"
87mebub9p4,0.257069621927,"{1: 1, 28: 1}"
lsw9q7uk0j,0.615367695957,"{3: 1, 28: 1}"
0d01nltbrs,0.687027310763,"{3: 1, 28: 1}"
a1vcnhxeij,0.90200615518,"{3: 1, 28: 1}"
6uh8zyj2gn,0.615367695957,"{1: 1, 28: 1}"
yuuqmid2rp,-0.101228452102,"{3: 1, 28: 1}"
om1ss59ys8,0.687027310763,"{3: 1, 28: 1}"


## Training a Local Outlier Factor (LOF) Model

Next, we train the **LOF model** by using this transformed **customer_data2** set.

In [11]:
model_lof = gl.anomaly_detection.local_outlier_factor.create(customer_data2, 
                                                             features = ['age', 'encoded_features'],
                                                             threshold_distances=True)

Starting brute force nearest neighbors model training.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

...

| 124654       | 1.6e+10 | 99.9783     | 1h 8m        |

| 124664       | 1.6e+10 | 99.9864     | 1h 8m        |

| 124675       | 1.6e+10 | 99.9952     | 1h 8m        |

| Done         |         | 100         | 1h 8m        |

+--------------+---------+-------------+--------------+

In [12]:
model_lof.save('./model_lof')

In [13]:
print 'The LOF model has been trained with the following options:'
print '-------------------------------------------------------------'
print model_lof.get_current_options()

The LOF model has been trained with the following options:
-------------------------------------------------------------
{'distance': [[['encoded_features'], 'jaccard', 1.0], [['age'], 'euclidean', 1.0]], 'verbose': True, 'num_neighbors': 5, 'threshold_distances': True}


Note that the model can automatically choose a suitable metric for the data type of the features we have available. Here, a composite distance of a **`'jaccard'`** and **`'euclidean'` metric** has been chosen for the **`'encoded_features'`** and the **`'age'`** columns respectively. Both these two metrics have been weighted with `1.0`. 

If we want what has been built by the model internally we can simply write:

In [14]:
print model_lof

Class                                   : LocalOutlierFactorModel

Schema
------
Number of examples                      : 124681
Number of feature columns               : 2
Number of neighbors                     : 5
Use thresholded distances               : True
Number of distance components           : 2
Row label name                          : row_id

Training summary
----------------
Total training time (seconds)           : 4115.3779

Accessible fields
-----------------
nearest_neighbors_model                 : Model used internally to compute nearest neighbors.
scores                                  : Local outlier factor for each row in the input dataset.


More importantly, here is the **SFrame** with the **LOF anomaly scores**:

In [15]:
model_lof['scores']

row_id,density,anomaly_score,neighborhood_radius
0,inf,nan,0.0
1,inf,nan,0.0
2,inf,nan,0.0
3,inf,nan,0.0
4,inf,nan,0.0
5,inf,nan,0.0
6,inf,nan,0.0
7,inf,nan,0.0
8,inf,nan,0.0
9,inf,nan,0.0


Firstly, note that the model worked successfully, scoring each of the 124,681 input rows. Secondly, the anomaly score for many observations in our **AirBnB dataset** is **`nan`** which indicates the point has many neighbors at exactly the same location, making the ratio of densities undefined. These points cannot be outliers.

However, for the problem at hand we are interested to find if any outliers exist and under what circumstances this happens. This is where the real business value exists!

## Using the LOF Model to detect anomalies

There are two common ways to detect which observations of your data set are anomalous or not:

**A.** Ask from the trained model to return the *k more anomalous observations*:
```
By applying the .topk() method of the model scores SFrame
```

In [16]:
top10_anomalies = model_lof['scores'].topk('anomaly_score', k=10)
top10_anomalies.print_rows(num_rows=10)

+--------+---------------+---------------+---------------------+
| row_id |    density    | anomaly_score | neighborhood_radius |
+--------+---------------+---------------+---------------------+
|  787   | 13.9548615034 |      inf      |   0.0716596148059   |
|  3678  | 13.9548615034 |      inf      |   0.0716596148059   |
|  5328  | 1.63764535897 |      inf      |    0.666666666667   |
|  6528  | 2.09512063742 |      inf      |    0.666666666667   |
|  8788  | 13.9548615034 |      inf      |   0.0716596148059   |
|  9626  | 13.9548615034 |      inf      |   0.0716596148059   |
| 10083  |      1.5      |      inf      |    0.666666666667   |
| 10727  | 13.9548615034 |      inf      |   0.0716596148059   |
| 10765  | 13.9548615034 |      inf      |   0.0716596148059   |
| 11038  |      1.5      |      inf      |    0.666666666667   |
+--------+---------------+---------------+---------------------+
[10 rows x 4 columns]



Note that the anomaly scores for these points are infinite, which happens when a point is next to several identical points, but is not itself a member of that bunch. 
```
But how this can be possible???
```
These points are certainly anomalous, but our specific choice of k was arbitrary and excluded many points that are also likely anomalous.

**B.** Choose a *threshold*, either from domain knowledge or scientific expertise in order to find the anomalous observations in your data set:
```
observations with 'anomaly_score' greater than this 'threshold' will be the anomalous ones.
```
Of course, a closer look at the distribution of the `anomaly_scores` may help us a lot with this decision.

In [17]:
anomaly_scores_sketch = model_lof['scores']['anomaly_score'].sketch_summary()
print anomaly_scores_sketch


+--------------------+--------+----------+
|        item        | value  | is exact |
+--------------------+--------+----------+
|       Length       | 124681 |   Yes    |
|        Min         | 0.865  |   Yes    |
|        Max         |  inf   |   Yes    |
|        Mean        |  nan   |   Yes    |
|        Sum         |  inf   |   Yes    |
|      Variance      |  nan   |   Yes    |
| Standard Deviation |  nan   |   Yes    |
|  # Missing Values  |   0    |   Yes    |
|  # unique values   |  578   |    No    |
+--------------------+--------+----------+

Most frequent items:
+-------+-----+-----+------+----------------+----------------+----------------+
| value | 1.0 | inf | 1.08 | 0.966666666667 | 0.942857142857 | 0.933333333333 |
+-------+-----+-----+------+----------------+----------------+----------------+
| count | 643 | 196 |  28  |       24       |       23       |       19       |
+-------+-----+-----+------+----------------+----------------+----------------+
+----------------+

In [18]:
threshold = anomaly_scores_sketch.quantile(0.9)
anomalies_mask = model_lof['scores']['anomaly_score'] >= threshold
anomalies = model_lof['scores'][anomalies_mask]
print 'Threshold: %.5f' % threshold, '\nNumber of Anomalies: %d' % len(anomalies)

Threshold: inf 
Number of Anomalies: 196


In [19]:
anomalies.print_rows(num_rows=10)

+--------+---------------+---------------+---------------------+
| row_id |    density    | anomaly_score | neighborhood_radius |
+--------+---------------+---------------+---------------------+
|  787   | 13.9548615034 |      inf      |   0.0716596148059   |
|  3678  | 13.9548615034 |      inf      |   0.0716596148059   |
|  5328  | 1.63764535897 |      inf      |    0.666666666667   |
|  6528  | 2.09512063742 |      inf      |    0.666666666667   |
|  8788  | 13.9548615034 |      inf      |   0.0716596148059   |
|  9626  | 13.9548615034 |      inf      |   0.0716596148059   |
| 10083  |      1.5      |      inf      |    0.666666666667   |
| 10727  | 13.9548615034 |      inf      |   0.0716596148059   |
| 10765  | 13.9548615034 |      inf      |   0.0716596148059   |
| 11038  |      1.5      |      inf      |    0.666666666667   |
+--------+---------------+---------------+---------------------+
[196 rows x 4 columns]



Finally, we can filter out the **`customer_data2`** set by the **`anomalies['row_id']`** to obtain the original features of these anomalous data points in record.

In [22]:
#customer_data = customer_data.add_row_number(column_name='row_id')
anomalous_customer_data = customer_data.filter_by(anomalies['row_id'], 'row_id')
anomalous_customer_data.print_rows(num_rows=200)

+--------+------------+-----------+-------+----------+
| row_id |     id     |   gender  |  age  | language |
+--------+------------+-----------+-------+----------+
|  787   | w6i3ix717s |   OTHER   |  36.0 |    en    |
|  3678  | jwzspk0ipl |    MALE   |  39.0 |    zh    |
|  5328  | eqsihtnz34 |   FEMALE  |  36.0 |    hu    |
|  6528  | dyu0sssqo5 | -unknown- |  47.0 |    nl    |
|  8788  | 91vfcvol82 |    MALE   |  91.0 |    en    |
|  9626  | t6fvmrna0t |    MALE   |  98.0 |    en    |
| 10083  | n45ipduv9i |    MALE   |  28.0 |    fi    |
| 10727  | 9zhr7vpciy |    MALE   |  39.0 |    fr    |
| 10765  | lerui8bp4h |   FEMALE  |  88.0 |    en    |
| 11038  | h0cf46ubyt |    MALE   |  27.0 |    fi    |
| 12293  | unnvgq3efo |    MALE   |  40.0 |    pl    |
| 13926  | 1yoqktv6n6 |   OTHER   |  36.0 |    en    |
| 13980  | 2a9z5icq6y |    MALE   |  39.0 |    de    |
| 14044  | oyr9d8w1ig |   OTHER   |  39.0 |    en    |
| 15897  | lqf1twcvos |    MALE   | 101.0 |    en    |
| 17154  |